<a href="https://colab.research.google.com/github/chhezue/NH-Invest-BigData/blob/main/1_ETF_Refine_Asset_Price_Connection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df_etf = pd.read_csv('/content/gdrive/MyDrive/NH 공모전/data/Yield_Week_NH_CONTEST_STK_DT_QUT_30.csv', header=None)
df_gold = pd.read_csv('/content/gdrive/MyDrive/NH 공모전/data/gold.csv')
df_coin = pd.read_csv('/content/gdrive/MyDrive/NH 공모전/data/coin.csv')
df_exchange = pd.read_csv('/content/gdrive/MyDrive/NH 공모전/data/exchange.csv')

# ETF 추출
- 종목명(tck_iem_cd), 주(Week), 변동률(yield_rate) 추출
- 변동률은 소수점 둘째 자리까지 반올림



In [ ]:
df_etf.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11
0,tck_iem_cd: AAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Week: 2024-05-27/2024-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bse_dt,iem_ong_pr,iem_hi_pr,iem_low_pr,iem_end_pr,bf_dd_cmp_ind_pr,bf_dd_cmp_ind_rt,acl_trd_qty,trd_cst,sll_cns_sum_qty,byn_cns_sum_qty,sby_bse_xcg_rt
3,2024-05-28,7.21,7.31,7.08,7.09,-0.12,-1.66,125993.0,893290.0,75160.0,50833.0,1365.4
4,2024-05-29,7.0,7.27,6.94,7.23,0.14,1.97,334259.0,2416693.0,148134.0,186125.0,1359.5
5,2024-05-30,7.32,7.73,7.27,7.71,0.48,6.64,333555.0,2571709.0,140370.0,193185.0,1364.8
6,2024-05-31,7.72,8.48,7.71,8.48,0.77,9.99,426990.0,3620875.0,228831.0,198159.0,1376.5
7,NaN,open,end,yield_rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,7.21,8.48,17.61442441054092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 결과를 저장할 딕셔너리 초기화
results = {}

# 현재 tck_iem_cd를 추적할 변수
current_tck_iem_cd = None

for index, row in df_etf.iterrows():
    # 'tck_iem_cd:'가 있는 행을 찾기
    if 'tck_iem_cd:' in str(row[0]):
        current_tck_iem_cd = row[0].strip().replace("tck_iem_cd: ", "")  # 현재 tck_iem_cd 저장 (문자열에서 제거)
        results[current_tck_iem_cd] = []  # 새로운 리스트 초기화

    # 'Week:'와 'yield_rate' 값을 찾기
    if current_tck_iem_cd is not None:
        if 'Week:' in str(row[0]):
            week_value = row[0].strip()  # 'Week' 값 저장
            results[current_tck_iem_cd].append({'Week': week_value})  # 'Week' 값 추가
        # 'yield_rate'가 포함된 행을 찾고 그 아래 행의 값을 가져오기
        elif 'yield_rate' in str(row.values):
            yield_rate_value = df_etf.iloc[index + 1, 3]  # yield_rate 다음 행의 4번째 열 (index 3)
            results[current_tck_iem_cd][-1]['yield_rate'] = yield_rate_value  # 마지막 'Week'에 yield_rate 추가

# 결과를 데이터프레임으로 변환
final_results = []

for tck, data in results.items():
    for entry in data:
        # yield_rate를 소수점 둘째 자리까지 반올림
        entry['yield_rate'] = round(float(entry['yield_rate']), 2)
        final_results.append({'tck_iem_cd': tck, 'Week': entry['Week'], 'yield_rate': entry['yield_rate']})

df_etf_final = pd.DataFrame(final_results)

# 결과 출력
print(df_etf_final)

      tck_iem_cd                         Week  yield_rate
0            AAN  Week: 2024-05-27/2024-06-02       17.61
1            AAN  Week: 2024-06-03/2024-06-09       -7.28
2            AAN  Week: 2024-06-10/2024-06-16       -5.28
3            AAN  Week: 2024-06-17/2024-06-23       -0.70
4            AAN  Week: 2024-06-24/2024-06-30        0.81
...          ...                          ...         ...
22790       ZYXI  Week: 2024-07-29/2024-08-04       -6.18
22791       ZYXI  Week: 2024-08-05/2024-08-11       -5.36
22792       ZYXI  Week: 2024-08-12/2024-08-18        2.37
22793       ZYXI  Week: 2024-08-19/2024-08-25        5.81
22794       ZYXI  Week: 2024-08-26/2024-09-01        2.96

[22795 rows x 3 columns]


# 금/비트코인/환율 시세 연결
- 금/비트코인 시세에 따라 ETF를 매수하므로 비트코인 1주차와 ETF 2주차를 비교

In [ ]:
df_coin.columns = df_coin.iloc[0]
df_coin = df_coin.drop(index=0)
df_coin['전주대비증감율'] = df_coin['전주대비증감율'].str.replace('%', '', regex=False)

In [ ]:
df_coin_new = df_coin[['거래일자(주별)', '전주대비증감율']]
df_coin_new

,거래일자(주별),전주대비증감율
1,Week: 2024-05-27/2024-06-02,-1.11
2,Week: 2024-06-03/2024-06-09,2.79
3,Week: 2024-06-10/2024-06-16,-4.32
4,Week: 2024-06-17/2024-06-23,-5.20
5,Week: 2024-06-24/2024-06-30,-0.79
6,Week: 2024-07-01/2024-07-07,-10.89
7,Week: 2024-07-08/2024-07-14,8.89
8,Week: 2024-07-15/2024-07-21,12.07
9,Week: 2024-07-22/2024-07-28,0.16
10,Week: 2024-07-29/2024-08-04,-14.87


In [ ]:
# "Week"로 시작하는 행과 그 옆의 숫자 추출
week_data = []

for i in range(len(df_gold)):
    # "Week"로 시작하는 행 찾기
    if isinstance(df_gold.iloc[i, 0], str) and df_gold.iloc[i, 0].startswith('Week'):
        # 다음 행에서 숫자 추출
        week_value = df_gold.iloc[i, 1]  # 바로 옆의 숫자
        week_value = round(float(week_value), 2)
        week_data.append({
            '거래일자(주별)': df_gold.iloc[i, 0],
            '전주대비증감율': week_value
        })

# 새로운 데이터프레임 생성
df_gold_new = pd.DataFrame(week_data)

In [ ]:
df_gold_new = df_gold_new.drop(index=13)
df_gold_new

,거래일자(주별),전주대비증감율
0,Week: 2024-05-27/2024-06-02,1.42
1,Week: 2024-06-03/2024-06-09,1.25
2,Week: 2024-06-10/2024-06-16,0.33
3,Week: 2024-06-17/2024-06-23,2.05
4,Week: 2024-06-24/2024-06-30,-0.77
5,Week: 2024-07-01/2024-07-07,1.48
6,Week: 2024-07-08/2024-07-14,0.71
7,Week: 2024-07-15/2024-07-21,0.76
8,Week: 2024-07-22/2024-07-28,-1.56
9,Week: 2024-07-29/2024-08-04,2.83


In [ ]:
df_exchange.columns = df_exchange.iloc[0]
df_exchange = df_exchange.drop(index=0)
df_exchange['전주대비증감율'] = df_exchange['전주대비증감율'].str.replace('%', '', regex=False)

In [ ]:
# "Week"로 시작하는 행과 그 옆의 숫자 추출
week_data = []

for i in range(len(df_exchange)):
    # "Week"로 시작하는 행 찾기
    if isinstance(df_exchange.iloc[i, 0], str) and df_exchange.iloc[i, 0].startswith('Week'):
        # 다음 행에서 숫자 추출
        week_value = df_exchange.iloc[i, 1]  # 바로 옆의 숫자
        week_value = round(float(week_value), 2)
        week_data.append({
            '거래일자(주별)': df_exchange.iloc[i, 0],
            '전주대비증감율': week_value
        })

# 새로운 데이터프레임 생성
df_exchange_new = pd.DataFrame(week_data)
df_exchange_new

,거래일자(주별),전주대비증감율
0,Week: 2024-05-27/2024-06-02,0.81
1,Week: 2024-06-03/2024-06-09,-0.73
2,Week: 2024-06-10/2024-06-16,0.30
3,Week: 2024-06-17/2024-06-23,0.44
4,Week: 2024-06-24/2024-06-30,0.01
5,Week: 2024-07-01/2024-07-07,-4.00
6,Week: 2024-07-08/2024-07-14,0.08
7,Week: 2024-07-15/2024-07-21,0.26
8,Week: 2024-07-22/2024-07-28,-0.09
9,Week: 2024-07-29/2024-08-04,-1.41


In [ ]:
# 금의 주차와 증감률을 저장할 새로운 열 생성
df_etf_final['금_주차'] = None
df_etf_final['금_증감률'] = None

# 금 데이터프레임의 인덱스 초기화
gold_index = 0

# 모든 종목에 대해 금의 주차와 증감률을 저장
for i in range(len(df_etf_final)):
    # 티커가 변경될 경우 gold_index를 초기화
    if i > 0 and df_etf_final.at[i, 'tck_iem_cd'] != df_etf_final.at[i - 1, 'tck_iem_cd']:
        gold_index = 0  # 새로운 티커 시작 시 인덱스 초기화

    # 금 정보를 저장할 조건
    if gold_index < len(df_gold_new):
        # 금의 정보는 ETF의 다음 주차에 저장
        if i < len(df_etf_final) - 1 and df_etf_final.at[i, 'tck_iem_cd'] == df_etf_final.at[i + 1, 'tck_iem_cd']:
            df_etf_final.at[i + 1, '금_주차'] = df_gold_new.at[gold_index, '거래일자(주별)']
            df_etf_final.at[i + 1, '금_증감률'] = df_gold_new.at[gold_index, '전주대비증감율']

        # 금의 인덱스를 증가시켜 다음 주 정보로 이동
        gold_index += 1

In [ ]:
df_etf_final.head(50)

,tck_iem_cd,Week,yield_rate,금_주차,금_증감률
0,AAN,Week: 2024-05-27/2024-06-02,17.61,None,None
1,AAN,Week: 2024-06-03/2024-06-09,-7.28,Week: 2024-05-27/2024-06-02,1.42
2,AAN,Week: 2024-06-10/2024-06-16,-5.28,Week: 2024-06-03/2024-06-09,1.25
3,AAN,Week: 2024-06-17/2024-06-23,-0.70,Week: 2024-06-10/2024-06-16,0.33
4,AAN,Week: 2024-06-24/2024-06-30,0.81,Week: 2024-06-17/2024-06-23,2.05
5,AAN,Week: 2024-07-01/2024-07-07,0.00,Week: 2024-06-24/2024-06-30,-0.77
6,AAN,Week: 2024-07-08/2024-07-14,0.10,Week: 2024-07-01/2024-07-07,1.48
7,AAN,Week: 2024-07-15/2024-07-21,-0.50,Week: 2024-07-08/2024-07-14,0.71
8,AAN,Week: 2024-07-22/2024-07-28,0.10,Week: 2024-07-15/2024-07-21,0.76
9,AAN,Week: 2024-07-29/2024-08-04,0.70,Week: 2024-07-22/2024-07-28,-1.56


In [ ]:
# '코인_증감률' 열을 None으로 초기화
df_etf_final['코인_증감률'] = None

# 모든 행에 대해 코인 증감률을 저장
for i in range(len(df_etf_final)):
    # 금의 주차가 None이 아닐 때만 진행
    if df_etf_final.at[i, '금_주차'] is not None:
        # 코인 데이터프레임에서 거래일자가 금의 주차와 같은지 확인
        matching_rows = df_coin_new[df_coin_new['거래일자(주별)'] == df_etf_final.at[i, '금_주차']]

        # 일치하는 행이 있는 경우, 증감률 저장
        if not matching_rows.empty:
            df_etf_final.at[i, '코인_증감률'] = matching_rows.iloc[0]['전주대비증감율']

In [ ]:
df_etf_final['환율_증감률'] = None

for i in range(len(df_etf_final)):
    if df_etf_final.at[i, '금_주차'] is not None:
        matching_rows = df_exchange_new[df_exchange_new['거래일자(주별)'] == df_etf_final.at[i, '금_주차']]
        if not matching_rows.empty:
            df_etf_final.at[i, '환율_증감률'] = matching_rows.iloc[0]['전주대비증감율']

In [ ]:
df_etf_final.head(50)

,tck_iem_cd,Week,yield_rate,금_주차,금_증감률,코인_증감률,환율_증감률
0,AAN,Week: 2024-05-27/2024-06-02,17.61,None,None,None,None
1,AAN,Week: 2024-06-03/2024-06-09,-7.28,Week: 2024-05-27/2024-06-02,1.42,-1.11,0.81
2,AAN,Week: 2024-06-10/2024-06-16,-5.28,Week: 2024-06-03/2024-06-09,1.25,2.79,-0.73
3,AAN,Week: 2024-06-17/2024-06-23,-0.70,Week: 2024-06-10/2024-06-16,0.33,-4.32,0.3
4,AAN,Week: 2024-06-24/2024-06-30,0.81,Week: 2024-06-17/2024-06-23,2.05,-5.20,0.44
5,AAN,Week: 2024-07-01/2024-07-07,0.00,Week: 2024-06-24/2024-06-30,-0.77,-0.79,0.01
6,AAN,Week: 2024-07-08/2024-07-14,0.10,Week: 2024-07-01/2024-07-07,1.48,-10.89,-4.0
7,AAN,Week: 2024-07-15/2024-07-21,-0.50,Week: 2024-07-08/2024-07-14,0.71,8.89,0.08
8,AAN,Week: 2024-07-22/2024-07-28,0.10,Week: 2024-07-15/2024-07-21,0.76,12.07,0.26
9,AAN,Week: 2024-07-29/2024-08-04,0.70,Week: 2024-07-22/2024-07-28,-1.56,0.16,-0.09


# 데이터프레임 정리/결측치 확인

In [ ]:
# 열 순서 변경
df_etf_final = df_etf_final[['tck_iem_cd', 'Week', 'yield_rate', '금_주차', '금_증감률', '코인_증감률', '환율_증감률']]

# 결과 확인
df_etf_final.head(50)

,tck_iem_cd,Week,yield_rate,금_주차,금_증감률,코인_증감률,환율_증감률
0,AAN,Week: 2024-05-27/2024-06-02,17.61,None,None,None,None
1,AAN,Week: 2024-06-03/2024-06-09,-7.28,Week: 2024-05-27/2024-06-02,1.42,-1.11,0.81
2,AAN,Week: 2024-06-10/2024-06-16,-5.28,Week: 2024-06-03/2024-06-09,1.25,2.79,-0.73
3,AAN,Week: 2024-06-17/2024-06-23,-0.70,Week: 2024-06-10/2024-06-16,0.33,-4.32,0.3
4,AAN,Week: 2024-06-24/2024-06-30,0.81,Week: 2024-06-17/2024-06-23,2.05,-5.20,0.44
5,AAN,Week: 2024-07-01/2024-07-07,0.00,Week: 2024-06-24/2024-06-30,-0.77,-0.79,0.01
6,AAN,Week: 2024-07-08/2024-07-14,0.10,Week: 2024-07-01/2024-07-07,1.48,-10.89,-4.0
7,AAN,Week: 2024-07-15/2024-07-21,-0.50,Week: 2024-07-08/2024-07-14,0.71,8.89,0.08
8,AAN,Week: 2024-07-22/2024-07-28,0.10,Week: 2024-07-15/2024-07-21,0.76,12.07,0.26
9,AAN,Week: 2024-07-29/2024-08-04,0.70,Week: 2024-07-22/2024-07-28,-1.56,0.16,-0.09


In [ ]:
# NaN 값 확인
nan_rows = df_etf_final[df_etf_final.isna().any(axis=1)]
nan_rows

,tck_iem_cd,Week,yield_rate,금_주차,금_증감률,코인_증감률,환율_증감률
0,AAN,Week: 2024-05-27/2024-06-02,17.61,None,None,None,None
14,AAON,Week: 2024-05-27/2024-06-02,-4.09,None,None,None,None
28,AAPB,Week: 2024-05-27/2024-06-02,0.61,None,None,None,None
42,AAT,Week: 2024-05-27/2024-06-02,2.06,None,None,None,None
56,ABCB,Week: 2024-05-27/2024-06-02,4.02,None,None,None,None
...,...,...,...,...,...,...,...
22725,ZUMZ,Week: 2024-05-27/2024-06-02,0.75,None,None,None,None
22739,ZURA,Week: 2024-05-27/2024-06-02,-5.06,None,None,None,None
22753,ZVRA,Week: 2024-05-27/2024-06-02,-0.43,None,None,None,None
22767,ZYME,Week: 2024-05-27/2024-06-02,-3.64,None,None,None,None


In [ ]:
df_etf_final.to_csv('df_etf_final.csv', index=False, encoding='utf-8-sig')
print("파일 저장 완료")

파일 저장 완료
